In [ ]:
from functools import partial
import numpy as np 
import scipy

from pyscf import gto, scf

np.set_printoptions(8)
np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])  

In [ ]:
mol = gto.Mole()
# mol.atom = """
# O 0.00000000 0.00000000 0.10683000
# H 0.00000000 0.78517800 -0.42731900
# H 0.00000000 -0.78517800 -0.42731900
# """
mol.atom = """
Li 1.60 0.0 0.0 
H 0.0 0.0 0.0
"""
# mol.atom = """
# H 0.75 0.0 0.0 
# H 0.0 0.0 0.0
# """

mol.basis = "6-31G"
mol.verbose = 5
mol.build()
scf_eng = scf.RHF(mol)
scf_eng.conv_tol = 1e-6
scf_eng.conv_tol_grad = 1e-6
scf_eng.max_cycle = 100
scf_eng.kernel()
scf_eng.converged

In [ ]:
def read_integral(filename):
    res = []
    with open(filename) as f:
        for line in f:
            if len(line) != 0:
                res.append(float(line))
    return np.array(res)

sorb = 11

s_ovlp = read_integral("./integral/S_LiH.txt").reshape(sorb, sorb)
h1e = read_integral("./integral/Hcore_LiH.txt").reshape(sorb, sorb)
h2e = read_integral("./integral/LiH.txt").reshape(sorb, sorb, sorb, sorb)

# s_ovlp = read_integral("./integral/S_H2.txt").reshape(sorb, sorb)
# h1e = read_integral("./integral/Hcore_H2.txt").reshape(sorb, sorb)
# h2e = read_integral("./integral/H2.txt").reshape(sorb, sorb, sorb, sorb)


s_ovlp_1 = mol.intor("int1e_ovlp")
h1e_1 = mol.intor("int1e_nuc") + mol.intor("int1e_kin")
h2e_1 = mol.intor("int2e")

print(np.abs(h1e - h1e_1).sum())
print(np.abs(h2e - h2e).sum())
print(np.abs(s_ovlp - s_ovlp).sum())

In [ ]:
def write_integral(filename):
    with open(filename, "w") as f:
        for i in range(len(h1e_1.reshape(-1))):
            f.write(f"{h1e_1.reshape(-1)[i]:.18f} \n")

# write_integral("./integral/Hcore_H2-pyscf.txt")
write_integral("./integral/Hcore_LiH-pyscf.txt")

In [ ]:
s_ovlp = mol.intor("int1e_ovlp")
h1e = mol.intor("int1e_nuc") + mol.intor("int1e_kin")
h2e = mol.intor("int2e")
# print(mol.intor("int2e_spinor").shape, h1e.shape)
# X = np.linalg.inv(np.linalg.cholesky(S).T)

print(f"ovlp: {s_ovlp}")
natm = mol.natm
nmo = nao = mol.nao
nocc = mol.nelec[0]
so = slice(0, nocc)
# μ(mu), ν(nu), κ(kappa), λ(lambda), σ(sigma)为原子轨道
# i, j, k, j 为分子轨道
# a, b, c, d 为非占据轨道
# p, q, r, s, m  全分子轨道
# print(s_ovlp)
# print(h1e)


X = scipy.linalg.fractional_matrix_power(s_ovlp, -0.5)

import torch
value, Vector = torch.linalg.eigh(torch.from_numpy(s_ovlp))
s_diag = torch.diag(value**(-0.5))
# print(s_diag)
# Vector @ s_diag.numpy() @ Vector.T , X
X = Vector @ s_diag.numpy() @ Vector.T

In [ ]:
#calculate nucleus energy 
Z_A = mol.atom_charges()
r_A = mol.atom_coords()

r_AB = np.zeros((mol.natm, mol.natm))
for i in range(mol.natm):
    for j in range(mol.natm):
        if i != j:
            r_AB[i, j] = np.linalg.norm(r_A[i] - r_A[j])
        else:
            r_AB[i, j] = np.infty 
E_nuc = 0.5 * np.einsum("A, B , AB ->", Z_A, Z_A, 1/r_AB)
print(f"Nucleus energy is  {E_nuc:8f} {mol.energy_nuc():8f} a.u. ")

In [ ]:
D = h1e
# print(h1e)
D_old = np.zeros((nao, nao))
count = 0 

print(D)

coeff_matrix = np.zeros((nao, nao))
# while (not np.allclose(D, D_old)):
while (True and count < 20):
    # if count > 3:
    #     raise ValueError("SCF does not converge")

    count += 1
    D_old = D
    # print(f"D_old\n: {D}")
    # Fock F_uv = H_uv + P_kl(uv|kl) - 0.5 *P_kl(ul|kv)
    fock_matrix = h1e + np.einsum("uvkl, kl ->uv", h2e, D) - 0.5 * np.einsum("ulkv, kl -> uv", h2e, D) 
    #Fock^{prime} F'= X.T @ F @ X 
    # print(f"fock:\n {fock_matrix}")
    
    # print(f"X:\n {X}")
    # break
    
    fock_matrix_p = X.T @ fock_matrix @ X
    
    # F'C' = C'e 
    eig_val, eig_vec = np.linalg.eigh(fock_matrix_p)
    coeff_matrix = X @ eig_vec
    # print(f"C_new:\n {coeff_matrix}")
    
    # D_{uv} = 2C_{ui}C_{vi}
    # D = 2 * coeff_matrix[:, so] @ coeff_matrix[:, so].T  
    D = 2 * np.einsum("ui, vi->uv", coeff_matrix[:, so], coeff_matrix[:, so])
    E_elec = (h1e * D).sum() + 0.5 * np.einsum("uvkl, uv, kl ->", h2e, D, D) - 0.25 * np.einsum("ulkv, uv, kl ->", h2e, D, D)
    E_elec1 = np.einsum("vu, uv->", D, (h1e + fock_matrix)) * 0.5

    print(f'E_elec: {E_elec:.10f}, E_elec1: {E_elec1:.10f}')
    # print(f"D-new:\n {D}")


# E_elec = 0.5 *\sum_u\sum_v P_vu(H_uv + F_uv)
# F_uv = H_uv + + P_kl(uv|kl) - 0.5 *P_kl(ul|kv)
E_elec = (h1e * D).sum() + 0.5 * np.einsum("uvkl, uv, kl ->", h2e, D, D) - 0.25 * np.einsum("ulkv, uv, kl ->", h2e, D, D)
E_tot = E_elec + E_nuc 

print(coeff_matrix.shape)
print(coeff_matrix[:, so].shape)

print(f"SCF Converged in  {count} loops")
print(f"Electronic energy {E_elec} a.u.")
print(f"Total energy     {E_tot} a.u.")

print(f"Pyscf energy is {scf_eng.e_tot} a.u. ")